In [ ]:
import numpy as np
import xarray as xr
import os
import pandas as pd
import glob

In [ ]:
import sys ; sys.path.append("../..")
from ppe_tools import *

In [ ]:
# set directories
exp_dir = '/glade/work/linnia/calLMIP/'

### 1. Define Parameters and Ranges

In [4]:
csv='calLMIP_ppe_paramranges_01132026.csv'
df = pd.read_csv(csv)
df_main = df[df['include'] == 1]
df_flagged = df[df['include'] == 0]
params=df_main['param'].values
flags = df_main['flag']

In [5]:
# grouped parameters
KCN=['kc_nonmyc','kn_nonmyc','akc_active','akn_active','ekc_active','ekn_active']
ACCLIM_SF=['vcmaxse_sf','jmaxse_sf','tpuse_sf']
TAU = ['bgc_tau_s1','bgc_tau_s2','bgc_tau_s3']
RF_LS = ['bgc_rf_l1s1','bgc_rf_l2s1','bgc_rf_l3s2']
RF_SS = ['bgc_rf_s2s1', 'bgc_rf_s2s3', 'bgc_rf_s3s1']
RF_CWD = ['rf_cwdl2', 'bgc_rf_cwdl3']

flagged_dict = {'KCN':KCN,'ACCLIM_SF':ACCLIM_SF,'TAU':TAU,'RF_LS':RF_LS,'RF_SS':RF_SS,'RF_CWD':RF_CWD}

### 2. Build Parameter Dictionary

In [6]:
def get_minmax(df,p):
    ix       = df['param']==p
    minval   = df['min'][ix].values[0]
    maxval   = df['max'][ix].values[0]
    pftmin   = df['pft_mins'][ix].values[0]
    pftmax   = df['pft_maxs'][ix].values[0]
    thisloc  = df['loc'][ix].values[0]

    needs_pft = (minval=='pft')
    if needs_pft:
        thismin = np.zeros((79))
        thismin[0:17] = np.fromstring(pftmin, dtype='float', sep=',')
    elif 'percent' in minval:
        thismin = minval
    else:
        thismin = np.array(float(minval))

    needs_pft = (maxval=='pft')
    if needs_pft:
        thismax = np.zeros((79))
        thismax[0:17] = np.fromstring(pftmax, dtype='float', sep=',')
    elif 'percent' in maxval:
        thismax = maxval
    else:
        thismax = np.array(float(maxval))

    return thismin, thismax, thisloc

In [7]:
lhcs = {}
for param,flag in zip(params,flags):
    
    if not pd.notnull(flag):
        thismin,thismax,thisloc = get_minmax(df,param)
        lhcs[param]={'min':thismin,'max':thismax,'loc':thisloc,'flagged':[]}
    else:
        flagged={}
        for p in flagged_dict[param]:
            thismin,thismax,thisloc = get_minmax(df_flagged,p)
            flagged[p]={'min':thismin,'max':thismax,'loc':thisloc}
        lhcs[param]={'min':[],'max':[],'loc':thisloc,'flagged':flagged}

### 4. Create the Ensemble Object

In [8]:
# 4a. instantiate the Ensemble object
basefile = '/glade/campaign/cesm/cesmdata/inputdata/lnd/clm2/paramdata/ctsm60_params.c251211.nc'
pdir = exp_dir + 'paramfiles/'
ndir = exp_dir + 'namelist_mods/'
x    = Ensemble(basefile,pdir,ndir)

# 4b. a the new ensemble members
prefix = 'lhc'   #make your own prefix
nextnum=1
n_samples=20  # set the number of ensemble members 
x.add_lhcs(lhcs,prefix,nextnum,n_samples,lhc=None) # for new LHC sample set lhc=None

In [9]:
pft_params = ['kmax','medlynintercept','r_mort','jmaxb0','slatop','medlynslope',
              'froot_leaf','leafcn','leaf_long']

for m in range(x.nmemb):
    for p in pft_params:
        if (p=='kmax') | (p=='psi50'):
            c3g = x.members[m].paramdict[p].value[:,13]
            x.members[m].paramdict[p].value[:,15] = c3g
            #x.members[m].paramdict[p].value[:,16] = c3g
        else:
            c3g = x.members[m].paramdict[p].value[13]
            x.members[m].paramdict[p].value[15] = c3g
            #x.members[m].paramdict[p].value[16] = c3g

### 5. Write the param_files

In [10]:
paramset_file = exp_dir + 'calLMIP_lhc_01132026.txt'
x.write(lhcfile=paramset_file)

In [11]:
%%bash -s "$paramset_file"
# Check that the file was created
paramset_file=$1 

stat $paramset_file

  File: /glade/work/linnia/calLMIP/calLMIP_lhc_01132026.txt
  Size: 7694      	Blocks: 32         IO Block: 8388608 regular file
Device: 50h/80d	Inode: 5263706789  Links: 1
Access: (0644/-rw-r--r--)  Uid: (35240/  linnia)   Gid: ( 1000/    ncar)
Access: 2026-01-13 14:44:29.727791000 -0700
Modify: 2026-01-13 14:44:29.728351934 -0700
Change: 2026-01-13 14:44:29.728351934 -0700
 Birth: -
